# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk

import sys
import os
import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV


from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')

df = pd.read_sql("SELECT * FROM InsertTableName", engine)


In [3]:
df.describe()

id       related       request         offer   aid_related  \
count  26215.000000  26215.000000  26215.000000  26215.000000  26215.000000   
mean   15224.871333      0.766470      0.170666      0.004501      0.414267   
std     8827.053788      0.423085      0.376224      0.066941      0.492604   
min        2.000000      0.000000      0.000000      0.000000      0.000000   
25%     7446.500000      1.000000      0.000000      0.000000      0.000000   
50%    15663.000000      1.000000      0.000000      0.000000      0.000000   
75%    22924.500000      1.000000      0.000000      0.000000      1.000000   
max    30265.000000      1.000000      1.000000      1.000000      1.000000   

       medical_help  medical_products  search_and_rescue      security  \
count  26215.000000      26215.000000       26215.000000  26215.000000   
mean       0.079496          0.050086           0.027618      0.017967   
std        0.270517          0.218126           0.163878      0.132833   
min        0.000000          0.000000           0.000000      0.000000   
25%        0.000000          0.000000           0.000000      0.000000   
50%        0.000000          0.000000           0.000000      0.000000   
75%        0.000000          0.000000           0.000000      0.000000   
max        1.000000          1.000000           1.000000      1.000000   

           military      ...         aid_centers  other_infrastructure  \
count  26215.000000      ...        26215.000000          26215.000000   
mean       0.032806      ...            0.011787              0.043906   
std        0.178131      ...            0.107929              0.204890   
min        0.000000      ...            0.000000              0.000000   
25%        0.000000      ...            0.000000              0.000000   
50%        0.000000      ...            0.000000              0.000000   
75%        0.000000      ...            0.000000              0.000000   
max        1.000000      ...            1.000000              1.000000   

       weather_related        floods         storm          fire  \
count     26215.000000  26215.000000  26215.000000  26215.000000   
mean          0.278352      0.082205      0.093191      0.010757   
std           0.448196      0.274682      0.290705      0.103160   
min           0.000000      0.000000      0.000000      0.000000   
25%           0.000000      0.000000      0.000000      0.000000   
50%           0.000000      0.000000      0.000000      0.000000   
75%           1.000000      0.000000      0.000000      0.000000   
max           1.000000      1.000000      1.000000      1.000000   

         earthquake          cold  other_weather  direct_report  
count  26215.000000  26215.000000   26215.000000   26215.000000  
mean       0.093649      0.020217       0.052489       0.193591  
std        0.291345      0.140746       0.223015       0.395120  
min        0.000000      0.000000       0.000000       0.000000  
25%        0.000000      0.000000       0.000000       0.000000  
50%        0.000000      0.000000       0.000000       0.000000  
75%        0.000000      0.000000       0.000000       0.000000  
max        1.000000      1.000000       1.000000       1.000000  

[8 rows x 36 columns]

In [4]:
X = df['message']

# categories are all columns after the 4th column
Y = df.iloc[:,4:]

### 2. Tokenization function to process  text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build Machine learning pipeline
This machine learning pipeline takes in the `message` column as input, and outputs the classification results on the other 36 categories in the dataset using the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) classifier for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

In [8]:
# train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test  model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred = pipeline.predict(X_test)

In [10]:
# overall accuracy

accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# classification report on test data

print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values))

Accuracy 94.71% 

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      3998
               request       0.79      0.54      0.64       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.59      0.67      2164
          medical_help       0.63      0.26      0.37       435
      medical_products       0.60      0.32      0.42       279
     search_and_rescue       0.60      0.18      0.27       136
              security       0.13      0.02      0.04        96
              military       0.57      0.25      0.35       158
                 water       0.74      0.62      0.68       335
                  food       0.85      0.70      0.77       584
               shelter       0.77      0.56      0.65       468
              clothing       0.82      0.44      0.57        70
                 money       0.51      0.27      0.35       112
        missing_peopl

In [11]:
# test msg

msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = pipeline.predict(msg)
print(Y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'shelter' 'buildings' 'fire'
 'direct_report']


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# check pipeline parameters
# pipeline.get_params()

parameters = {
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__learning_rate': [0.1, 0.5, 1]

    } 


cv = GridSearchCV(estimator=pipeline, n_jobs = -1, param_grid=parameters)

In [15]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__learning_rate': [0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

Try:
Inherently multiclass: Naive Bayes, LDA and QDA, Decision Trees, Random Forests, Nearest Neighbors, setting multi_class='multinomial' in sklearn.linear_model.LogisticRegression.
Support multilabel: Decision Trees, Random Forests, Nearest Neighbors.
One-Vs-One: sklearn.svm.SVC.
One-Vs-All: all linear models except sklearn.svm.SVC.

In [16]:
Y_pred = cv.predict(X_test)

In [17]:
# overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# classification report on test data
print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values))

print("\nBest Parameters:", cv.best_params_)


Accuracy 94.55% 

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      3998
               request       0.78      0.48      0.60       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.55      0.63      2164
          medical_help       0.61      0.25      0.35       435
      medical_products       0.70      0.27      0.39       279
     search_and_rescue       0.69      0.15      0.24       136
              security       0.14      0.02      0.04        96
              military       0.53      0.22      0.31       158
                 water       0.74      0.58      0.65       335
                  food       0.82      0.66      0.73       584
               shelter       0.77      0.46      0.57       468
              clothing       0.76      0.46      0.57        70
                 money       0.54      0.29      0.37       112
        missing_peopl

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
# Create custom transformer

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [19]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(LinearSVC()))
])

In [20]:
parameters = {
        'clf__estimator__loss': ('hinge', 'squared_hinge'),
        'clf__estimator__C': (0.01, 0.5, 1.0)
    } 


cv = GridSearchCV(estimator=pipeline, n_jobs = -1, param_grid=parameters)

In [21]:
# train pipeline
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

In [22]:
# overall accuracy
accuracy = (Y_pred == Y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# classification report on test data
print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values))

print("\nBest Parameters:", cv.best_params_)

Accuracy 95.12% 

                        precision    recall  f1-score   support

               related       0.86      0.93      0.90      3998
               request       0.80      0.59      0.68       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.70      0.73      2164
          medical_help       0.65      0.23      0.34       435
      medical_products       0.72      0.25      0.37       279
     search_and_rescue       0.67      0.16      0.26       136
              security       0.00      0.00      0.00        96
              military       0.58      0.21      0.31       158
                 water       0.76      0.65      0.70       335
                  food       0.83      0.77      0.80       584
               shelter       0.81      0.56      0.66       468
              clothing       0.70      0.47      0.56        70
                 money       0.68      0.17      0.27       112
        missing_peopl

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
m = pickle.dumps('classifier.pkl')